In [114]:
from sqlalchemy import create_engine
import os

DB_USER = os.getenv("DB_USER", "root")
DB_PASSWORD = os.getenv("DB_PASSWORD", "root123")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "3306")
DB_NAME = os.getenv("DB_NAME", "smarthome_db")

DATABASE_URL = (
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    "?charset=utf8mb4"
)

engine = create_engine(
    DATABASE_URL,
    pool_pre_ping=True,       # keep connections alive
    pool_recycle=3600,        # recycle connections every hour
    pool_size=10,             # optional: connection pool size
    max_overflow=20,          # optional: extra temporary connections
    echo=False,               # show SQL queries when True
    future=True               # SQLAlchemy 2.0 style
)

In [115]:
import pandas as pd
query = '''
SELECT  variant_id, price
FROM `productvariants`;
'''
df_product_variant = pd.read_sql(query, engine)
df_product_variant.head()

,variant_id,price
0,35,6490000.0
1,36,7890000.0
2,37,6590000.0
3,38,7590000.0
4,39,14690000.0


In [116]:
import pandas as pd
# variant_id, price_at_event, click_counting, event_time, event_type
query = '''
SELECT variant_id, price_at_event as unitprice, click_counting as view, event_time, event_type
FROM `product_events`
WHERE event_type = 'view'
'''
df_product_view_events = pd.read_sql(query, engine)
df_product_view_events['event_time'] = pd.to_datetime(df_product_view_events['event_time'])
df_product_view_events['year'] = df_product_view_events['event_time'].dt.year
df_product_view_events['month'] = df_product_view_events['event_time'].dt.month
df_product_view_events = df_product_view_events.groupby(['variant_id','year', 'month']).agg({
    'view': 'sum',
})
df_product_view_events.head()

,,,view
variant_id,year,month,
37,2025,11,13
38,2025,11,2
39,2025,11,3
40,2025,11,2
47,2025,11,6


In [117]:
import pandas as pd
# variant_id, price_at_event, click_counting, event_time, event_type
query = '''
SELECT variant_id, price_at_event as unitprice, click_counting as add_to_cart, event_time, event_type 
FROM `product_events`
WHERE event_type = 'add_to_cart'
'''
df_product_add_to_cart_events = pd.read_sql(query, engine)
df_product_add_to_cart_events['event_time'] = pd.to_datetime(df_product_add_to_cart_events['event_time'])
df_product_add_to_cart_events['year'] = df_product_add_to_cart_events['event_time'].dt.year
df_product_add_to_cart_events['month'] = df_product_add_to_cart_events['event_time'].dt.month
df_product_add_to_cart_events = df_product_add_to_cart_events.groupby(['variant_id','year', 'month']).agg({
    'add_to_cart': 'sum',
})
df_product_add_to_cart_events.head()

,,,add_to_cart
variant_id,year,month,
37,2025,10,35
71,2025,10,1


In [118]:
import pandas as pd
# variant_id, quantity, price, total_price
query = '''
SELECT variant_id, quantity as orderquantity, price as unitprice, total_price as sales, created_at
FROM `orderitems`, `orders`
WHERE orderitems.order_id = orders.order_id
'''
df_orders = pd.read_sql(query, engine)
df_orders['created_at'] = pd.to_datetime(df_orders['created_at'])
df_orders['year'] = df_orders['created_at'].dt.year
df_orders['month'] = df_orders['created_at'].dt.month
df_orders = df_orders.groupby(['variant_id', 'year', 'month']).agg({
    'orderquantity': 'sum',
    'unitprice': 'mean',
    'sales': 'mean'

})
df_orders.head(n=-1)

orderquantity     unitprice         sales
variant_id year month                                           
35         2024 12                 1  6.990000e+06  6.990000e+06
           2025 1                  1  6.990000e+06  6.990000e+06
                4                  1  6.490000e+06  6.490000e+06
                5                  1  6.990000e+06  6.990000e+06
36         2024 12                 1  7.890000e+06  7.890000e+06
           2025 1                  1  7.890000e+06  7.890000e+06
                7                  2  7.890000e+06  1.578000e+07
37         2025 1                  1  7.590000e+06  7.590000e+06
                6                  1  7.490000e+06  7.490000e+06
                8                  7  7.208000e+06  1.032000e+07
38         2025 2                  2  8.885000e+06  8.885000e+06
                3                  1  9.380000e+06  9.380000e+06
                8                  2  8.590000e+06  8.590000e+06
39         2025 3                  1  1.559000e+07  1.559000e+07
                5                  1  1.609000e+07  1.609000e+07
                6                  1  1.469000e+07  1.469000e+07
                8                  4  1.591500e+07  1.591500e+07
40         2025 6                  3  1.872333e+07  1.872333e+07
                8                  1  1.914000e+07  1.914000e+07
47         2025 4                  1  5.900000e+06  5.900000e+06
49         2025 2                  3  6.790000e+06  1.003500e+07
                3                  2  7.090000e+06  1.418000e+07
                8                  1  6.490000e+06  6.490000e+06
50         2025 8                  4  1.299000e+07  1.299000e+07
52         2025 8                 14  1.077571e+07  1.077571e+07
53         2025 8                 19  9.990000e+06  2.676200e+07
56         2025 8                  1  3.200000e+07  3.200000e+07
58         2025 8                  2  1.259000e+07  1.259000e+07
59         2025 8                  1  1.399000e+07  1.399000e+07

In [119]:
df = df_orders.merge(
        df_product_view_events, 
        on=['variant_id', 'year', 'month'], 
        how='outer'
    ).merge(
        df_product_add_to_cart_events,
        on=['variant_id', 'year', 'month'],
        how='outer'
    )

df.head(n=-1)

orderquantity     unitprice         sales  view  \
variant_id year month                                                    
35         2024 12               1.0  6.990000e+06  6.990000e+06   NaN   
           2025 1                1.0  6.990000e+06  6.990000e+06   NaN   
                4                1.0  6.490000e+06  6.490000e+06   NaN   
                5                1.0  6.990000e+06  6.990000e+06   NaN   
36         2024 12               1.0  7.890000e+06  7.890000e+06   NaN   
           2025 1                1.0  7.890000e+06  7.890000e+06   NaN   
                7                2.0  7.890000e+06  1.578000e+07   NaN   
37         2025 1                1.0  7.590000e+06  7.590000e+06   NaN   
                6                1.0  7.490000e+06  7.490000e+06   NaN   
                8                7.0  7.208000e+06  1.032000e+07   NaN   
                10               NaN           NaN           NaN   NaN   
                11               NaN           NaN           NaN  13.0   
38         2025 2                2.0  8.885000e+06  8.885000e+06   NaN   
                3                1.0  9.380000e+06  9.380000e+06   NaN   
                8                2.0  8.590000e+06  8.590000e+06   NaN   
                11               NaN           NaN           NaN   2.0   
39         2025 3                1.0  1.559000e+07  1.559000e+07   NaN   
                5                1.0  1.609000e+07  1.609000e+07   NaN   
                6                1.0  1.469000e+07  1.469000e+07   NaN   
                8                4.0  1.591500e+07  1.591500e+07   NaN   
                11               NaN           NaN           NaN   3.0   
40         2025 6                3.0  1.872333e+07  1.872333e+07   NaN   
                8                1.0  1.914000e+07  1.914000e+07   NaN   
                11               NaN           NaN           NaN   2.0   
47         2025 4                1.0  5.900000e+06  5.900000e+06   NaN   
                11               NaN           NaN           NaN   6.0   
48         2025 11               NaN           NaN           NaN   6.0   
49         2025 2                3.0  6.790000e+06  1.003500e+07   NaN   
                3                2.0  7.090000e+06  1.418000e+07   NaN   
                8                1.0  6.490000e+06  6.490000e+06   NaN   
                11               NaN           NaN           NaN   6.0   
50         2025 8                4.0  1.299000e+07  1.299000e+07   NaN   
52         2025 8               14.0  1.077571e+07  1.077571e+07   NaN   
53         2025 8               19.0  9.990000e+06  2.676200e+07   NaN   
56         2025 8                1.0  3.200000e+07  3.200000e+07   NaN   
57         2025 11               NaN           NaN           NaN   1.0   
58         2025 8                2.0  1.259000e+07  1.259000e+07   NaN   
                11               NaN           NaN           NaN   2.0   
59         2025 8                1.0  1.399000e+07  1.399000e+07   NaN   
62         2025 11               NaN           NaN           NaN   4.0   
63         2025 11               NaN           NaN           NaN   5.0   
64         2025 8                1.0  5.088000e+06  5.088000e+06   NaN   
                11               NaN           NaN           NaN   3.0   
65         2025 11               NaN           NaN           NaN   1.0   
66         2025 10               NaN           NaN           NaN   2.0   
71         2025 10               NaN           NaN           NaN   NaN   
                11               NaN           NaN           NaN   7.0   
72         2025 11               NaN           NaN           NaN   1.0   
73         2025 11               NaN           NaN           NaN   3.0   

                       add_to_cart  
variant_id year month               
35         2024 12             NaN  
           2025 1              NaN  
                4              NaN  
                5              NaN  
36         2024 12        

In [120]:
df['best_seller_of_month'] = df.groupby(['year','month'])['orderquantity']\
                                         .transform(lambda x: x == x.max()).astype(int)

In [121]:
monthly = df.sort_values(by=['variant_id', 'year', 'month'])


monthly['orderquantity_next_month'] = monthly.groupby('variant_id')['orderquantity'].shift(-1)
monthly['view_next_month'] = monthly.groupby('variant_id')['view'].shift(-1)
monthly['add_to_cart_next_month'] = monthly.groupby('variant_id')['add_to_cart'].shift(-1)
monthly['best_seller_next_month'] = monthly.groupby('variant_id')['best_seller_of_month'].shift(-1)



monthly['add_to_cart_rate'] = monthly["add_to_cart"] / monthly["view"]
monthly["cart_to_order_rate"] = monthly["orderquantity"] / monthly["add_to_cart"]
monthly["conversion_rate"] = monthly["orderquantity"] / monthly["view"]
monthly["aov"] = monthly["sales"] / monthly["orderquantity"]
monthly["order_growth"] = monthly["orderquantity_next_month"] / monthly["orderquantity"]
monthly["view_growth"] = monthly["view_next_month"] / monthly["view"]
monthly["add_to_cart_growth"] = monthly["add_to_cart_next_month"] / monthly["add_to_cart"]


In [122]:
monthly["aov_lag_1"] = monthly.groupby("variant_id")["aov"].shift(1)
monthly["add_to_cart_lag_1"] = monthly.groupby("variant_id")["add_to_cart"].shift(1)
monthly["order_3m_avg"] = monthly.groupby("variant_id")["orderquantity"].rolling(3).mean().reset_index(level=0, drop=True)
monthly["order_6m_avg"] = monthly.groupby("variant_id")["orderquantity"].rolling(6).mean().reset_index(level=0, drop=True)
monthly["view_3m_avg"] = monthly.groupby("variant_id")["view"].rolling(3).mean().reset_index(level=0, drop=True)
monthly["orderquantity_lag_1"] = monthly.groupby("variant_id")["orderquantity"].shift(1)
monthly["orderquantity_lag_2"] = monthly.groupby("variant_id")["orderquantity"].shift(2)
monthly["orderquantity_lag_3"] = monthly.groupby("variant_id")["orderquantity"].shift(3)
monthly["view_lag_1"] = monthly.groupby("variant_id")["view"].shift(1)
monthly["add_to_cart_3m_avg"] = monthly.groupby("variant_id")["add_to_cart"].rolling(3).mean().reset_index(level=0, drop=True)


In [124]:
monthly.head(n=-1)

orderquantity     unitprice         sales  view  \
variant_id year month                                                    
35         2024 12               1.0  6.990000e+06  6.990000e+06   NaN   
           2025 1                1.0  6.990000e+06  6.990000e+06   NaN   
                4                1.0  6.490000e+06  6.490000e+06   NaN   
                5                1.0  6.990000e+06  6.990000e+06   NaN   
36         2024 12               1.0  7.890000e+06  7.890000e+06   NaN   
           2025 1                1.0  7.890000e+06  7.890000e+06   NaN   
                7                2.0  7.890000e+06  1.578000e+07   NaN   
37         2025 1                1.0  7.590000e+06  7.590000e+06   NaN   
                6                1.0  7.490000e+06  7.490000e+06   NaN   
                8                7.0  7.208000e+06  1.032000e+07   NaN   
                10               NaN           NaN           NaN   NaN   
                11               NaN           NaN           NaN  13.0   
38         2025 2                2.0  8.885000e+06  8.885000e+06   NaN   
                3                1.0  9.380000e+06  9.380000e+06   NaN   
                8                2.0  8.590000e+06  8.590000e+06   NaN   
                11               NaN           NaN           NaN   2.0   
39         2025 3                1.0  1.559000e+07  1.559000e+07   NaN   
                5                1.0  1.609000e+07  1.609000e+07   NaN   
                6                1.0  1.469000e+07  1.469000e+07   NaN   
                8                4.0  1.591500e+07  1.591500e+07   NaN   
                11               NaN           NaN           NaN   3.0   
40         2025 6                3.0  1.872333e+07  1.872333e+07   NaN   
                8                1.0  1.914000e+07  1.914000e+07   NaN   
                11               NaN           NaN           NaN   2.0   
47         2025 4                1.0  5.900000e+06  5.900000e+06   NaN   
                11               NaN           NaN           NaN   6.0   
48         2025 11               NaN           NaN           NaN   6.0   
49         2025 2                3.0  6.790000e+06  1.003500e+07   NaN   
                3                2.0  7.090000e+06  1.418000e+07   NaN   
                8                1.0  6.490000e+06  6.490000e+06   NaN   
                11               NaN           NaN           NaN   6.0   
50         2025 8                4.0  1.299000e+07  1.299000e+07   NaN   
52         2025 8               14.0  1.077571e+07  1.077571e+07   NaN   
53         2025 8               19.0  9.990000e+06  2.676200e+07   NaN   
56         2025 8                1.0  3.200000e+07  3.200000e+07   NaN   
57         2025 11               NaN           NaN           NaN   1.0   
58         2025 8                2.0  1.259000e+07  1.259000e+07   NaN   
                11               NaN           NaN           NaN   2.0   
59         2025 8                1.0  1.399000e+07  1.399000e+07   NaN   
62         2025 11               NaN           NaN           NaN   4.0   
63         2025 11               NaN           NaN           NaN   5.0   
64         2025 8                1.0  5.088000e+06  5.088000e+06   NaN   
                11               NaN           NaN           NaN   3.0   
65         2025 11               NaN           NaN           NaN   1.0   
66         2025 10               NaN           NaN           NaN   2.0   
71         2025 10               NaN           NaN           NaN   NaN   
                11               NaN           NaN           NaN   7.0   
72         2025 11               NaN           NaN           NaN   1.0   
73         2025 11               NaN           NaN           NaN   3.0   

                       add_to_cart  best_seller_of_month  \
variant_id year month                                      
35         2024 12             NaN                     1   
           2025 1              NaN                     1   
        

In [127]:
print(df.columns.tolist())

['orderquantity', 'unitprice', 'sales', 'view', 'add_to_cart', 'best_seller_of_month']
